# Imports

In [ ]:
from scipy.optimize import minimize
import time
import closely
import numpy as np
import pickle
from datetime import datetime
import math
from use_case import ClinicalTrial

ModuleNotFoundError: No module named 'closely'

# Solution

## QAOA

### constructing QUBO matrix

In [ ]:
def constuct_qubo_matrix(input):
    n = input.w.shape[0]
    n_squared = n * n
    Q = np.zeros((n, n))
    offset = 0
    # mean
    for w_index in range(0, 3):
        for i1 in range(0, n):
            for i2 in range(0, n):
                Q[i1, i2] += 4 * input.w[i1, w_index] * input.w[i2, w_index] / n_squared
                Q[i1, i1] -= 2 * input.w[i1, w_index] * input.w[i2, w_index] / n_squared
                Q[i2, i2] -= 2 * input.w[i1, w_index] * input.w[i2, w_index] / n_squared
                offset += input.w[i1, w_index] * input.w[i2, w_index] / n_squared
    # var i
    for w_index in range(0, 3):
        for i1 in range(0, n):
            for i2 in range(0, n):
                Q[i1, i2] += input.rho * 4 * input.w[i1, w_index] ** 2 * input.w[i2, w_index] ** 2 / n_squared
                Q[i1, i1] -= input.rho * 2 * input.w[i1, w_index] ** 2 * input.w[i2, w_index] ** 2 / n_squared
                Q[i2, i2] -= input.rho * 2 * input.w[i1, w_index] ** 2 * input.w[i2, w_index] ** 2 / n_squared
                offset += input.rho * input.w[i1, w_index] ** 2 * input.w[i2, w_index] ** 2 / n_squared
    # var i j
    for w_index1 in range(0, 3):
        for w_index2 in range(w_index1+1, 3):
            for i1 in range(0, n):
                for i2 in range(0, n):
                    Q[i1, i2] +=  2 * input.rho * 4 * input.w[i1, w_index1] * input.w[i1, w_index2] * input.w[i2, w_index1] * input.w[i2, w_index2] / n_squared
                    Q[i1, i1] -= 2 * input.rho * 2 * input.w[i1, w_index1] * input.w[i1, w_index2] * input.w[i2, w_index1] * input.w[i2, w_index2] / n_squared
                    Q[i2, i2] -= 2 * input.rho * 2 * input.w[i1, w_index1] * input.w[i1, w_index2] * input.w[i2, w_index1] * input.w[i2, w_index2] / n_squared
                    offset += 2 * input.rho * input.w[i1, w_index1] * input.w[i1, w_index2] * input.w[i2, w_index1] * input.w[i2, w_index2] / n_squared
    for i in range(0, n):
        for j in range(0, n):
            if i > j:
                Q[i, j] *= 2
            elif i < j:
                Q[i, j] = 0
    return Q[0:-1,0:-1], offset

def qubo_cost(x, Q, offset):
    return x.T @ Q @ x + offset

### Custom QAOA

##### qaoa circuit

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import XXPlusYYGate
from qiskit_aer import AerSimulator
from qiskit import transpile

def xy_ring_mixer(n_qubits, angle):
    circuit = QuantumCircuit(n_qubits)

    # ring
    pairs = [[i, i + 1] for i in range(n_qubits - 1)]
    pairs.append([n_qubits - 1, 0])

    for i, e in enumerate(pairs):
        circuit.append(XXPlusYYGate(0.5 * angle), e)

    return circuit

def initilize(n_qubits, bitarr):
    circuit = QuantumCircuit(n_qubits)
    if bitarr is None:
            for i in range(0, n_qubits, 2):
                circuit.x(i)
    else:
        for i in range(0, n_qubits):
            if bitarr[i] == 1:
                circuit.x(i)
    return circuit

def cost_layer(Q, param):
    n_qubits = Q.shape[0]
    circuit = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        w_i = 0.5 * np.sum(Q[:, i])

        if not math.isclose(w_i, 0, abs_tol=1e-7):
            circuit.rz(param * w_i, i)

        for j in range(i + 1, n_qubits):
            w_ij = 0.25 * Q[j][i]

            if not math.isclose(w_ij, 0, abs_tol=1e-7):
                circuit.cx(i, j)
                circuit.rz(param * w_ij, j)
                circuit.cx(i, j)
    return circuit

def qaoa_circuit(Q, mixer_layers, params, bitarr_init):
    n_qubits = Q.shape[0]
    circuit_length = params.shape[0] // 2
    cost_layer_params = params[:circuit_length]
    mixer_layer_params = params[circuit_length:]
    circuit = initilize(n_qubits, bitarr_init)
    for layer in range(0, cost_layer_params.shape[0]):
        circuit.compose(cost_layer(Q, cost_layer_params[layer]))
        for mixer_layer in range(mixer_layers):
            circuit.compose(xy_ring_mixer(n_qubits, mixer_layer_params[layer]), inplace=True)
    circuit.measure_all()
    return circuit

device = AerSimulator()

def run_qaoa_circuit(device, shots, Q, mixer_layers, params, bitarr_init):
    circuit = qaoa_circuit(Q, mixer_layers, params, bitarr_init)
    circuit = transpile(circuit, device)
    result = device.run(circuits=circuit, shots=shots).result()
    counts = result.get_counts(circuit)
    return counts

##### group helpers

In [ ]:
def group_complement(group):
    return 1 - group

def group_from_x(x):
    return np.append(x, 0)

def groups_from_x(x):
    group1 = group_from_x(x)
    group2 = group_complement(group1)
    return group1, group2

##### cost function

In [ ]:
output = None
def process_counts(counts, shots, Q, offset, is_qubo_cost):
    energy_expect = 0
    minimal_energy = None
    best_sol = None
    for bin_string, count in counts.items():
        x = (np.fromstring(bin_string, 'u1') - ord('0')).astype(int)
        if is_qubo_cost:
            cost = qubo_cost(x, Q, offset)
        else:
            group1, group2 = groups_from_x(x)
            cost = input.discrepancy(group1, group2)
        energy_expect += count * cost / shots
        if minimal_energy is None or cost < minimal_energy:
            minimal_energy = cost
            best_sol = x
    return energy_expect, minimal_energy, best_sol

def update_tracker(energy_expect, minimal_energy, best_sol, params, tracker, verbose):
    tracker["energy_expectations"].append(energy_expect)
    tracker["step_opt_energies"].append(minimal_energy)
    tracker["step_opt_sol"].append(best_sol)

    if verbose:
        print("Minimal energy:", minimal_energy)
        print("Energy expectation:", energy_expect)
        print("Parameters:", params)

    tracker.update({"count": tracker["count"] + 1})

    tracker["params"].append(params)

    if tracker['min_energy_expect'] is None or energy_expect < tracker['min_energy_expect']:
        tracker.update({"min_energy_expect": energy_expect})
        tracker.update({"opt_params": params})

    # store global optimum
    if len(tracker["global_opt_energies"]) == 0 or minimal_energy < tracker["global_opt_energies"][-1]:
        tracker["global_opt_energies"].append(minimal_energy)
        tracker.update({"global_opt_sol": best_sol})
        output = group_from_x(best_sol)
    else:
        tracker["global_opt_energies"].append(tracker["global_opt_energies"][-1])

def cost_function(params, Q, offset, mixer_layers, device, shots, tracker, bitarr_init, is_qubo_cost, verbose):
    if verbose:
        print("==================================" * 2)
        print("Calling the quantum circuit. Cycle:", tracker["count"])
    counts = run_qaoa_circuit(device, shots, Q, mixer_layers, params, bitarr_init)
    energy_expect, minimal_energy, best_sol = process_counts(counts, shots, Q, offset, is_qubo_cost)
    update_tracker(energy_expect, minimal_energy, best_sol, params, tracker, verbose)
    cost = energy_expect
    if verbose:
        print("Cost:", cost)
    return cost

##### optimization

In [ ]:
def optimize(params0, maxfev, opt_method, Q, offset, mixer_layers, device, shots, tracker, bitarr_init, is_qubo, verbose):
    bnds = [(0, 2 * np.pi) for _ in range(params0.shape[0])]
    options = {'disp': True, 'maxiter': maxfev}
    minimize(
        cost_function,
        params0,
        args=(Q, offset, mixer_layers, device, shots, tracker, bitarr_init, is_qubo, verbose),
        options=options,
        method=opt_method,
        bounds=bnds,
        tol=1e-6
    )

##### tracker

In [ ]:
def initialize_tracker():
    tracker = {
        'count': 0,                           # Elapsed optimization steps
        'step_opt_energies': [],              # Optimal energy at each step
        'global_opt_energies': [],            # Global optimal energy at each step
        'global_opt_sol': None,               # Global optimal bitstring
        'step_opt_sol': [],                   # Optimal bitstring at each step
        'energy_expectations': [],            # energy expectation at each step
        'params': [],                         # parameters at each step
        'opt_params': None,                   # Track optimal parameters
        'min_energy_expect': None,            # Global minumum energy expectation value reached
        'time': time.time()                          # Time of the optimization
    }
    return tracker

##### closest pairs

In [ ]:
def initial_partition_by_closest_pairs(input):
    points = input.w.copy()

    points_pos = {}
    for i in range(0, points.shape[0]):
        points_pos[tuple(points[i])] = i

    bitarr_init = np.empty(input.w.shape[0], dtype=int)
    for i in range(0, input.w.shape[0] // 2):
        pairs, distances = closely.solve(points, n=1)
        bitarr_init[points_pos[tuple(points[pairs[0, 0]])]] = 0
        bitarr_init[points_pos[tuple(points[pairs[0, 1]])]] = 1
        points = np.delete(points, [pairs[0, 0], pairs[0, 1]], axis=0)

    if bitarr_init[0] == 1:
        bitarr_init = 1 - bitarr_init
    #print('closest pair discrepancy =', input.discrepancy(bitarr_init, group_complement(bitarr_init)))
    bitarr_init = bitarr_init[1:]

    return bitarr_init

##### solve

In [ ]:
def solve(input):
    # hyperparams
    depth = 1
    mixer_layers = 16
    shots = 2500
    #device = AerSimulator()
    device = AerSimulator(method='statevector', device='GPU')
    bitarr_init = initial_partition_by_closest_pairs(input)
    is_qubo = False
    verbose = False
    maxfev = 20
    opt_method = 'COBYLA'
    n_iter = 3

    Q, offset = constuct_qubo_matrix(input)

    tracker = initialize_tracker()
    for _ in range(n_iter):
        tracker.update({"count": 0})
        # optimize
        params0 = np.random.uniform(0, 2 * np.pi, 2 * depth)
        optimize(params0, maxfev, opt_method, Q, offset, mixer_layers, device, shots, tracker, bitarr_init, is_qubo, verbose)

    end = time.time()
    tracker.update({"time": end - tracker["time"]})
    #print("time: ", tracker["time"])

    return tracker["global_opt_energies"][-1], tracker["global_opt_sol"]

##### solution

In [3]:
n_clusters = 5
cluster_size = input.w.shape[0] // n_clusters

In [ ]:
from k_means_constrained import KMeansConstrained
kmeans_c = KMeansConstrained(n_clusters=n_clusters, size_min=cluster_size, size_max=cluster_size, random_state=0)
kmeans_c.fit(input.w)
cluster_labels = kmeans_c.labels_

In [ ]:
np.random.seed(1)

In [ ]:
opt_group = np.empty(input.w.shape[0])
w_original = input.w
for i in range(0, n_clusters):
    indecies = (cluster_labels == i).nonzero()[0]
    input.w = w_original[cluster_labels == i]
    _, cluster_opt_sol = solve(input)
    cluster_opt_group = group_from_x(cluster_opt_sol)
    for j in range(0, cluster_opt_group.shape[0]):
        opt_group[indecies[j]] = cluster_opt_group[j]
input.w = w_original

In [5]:
print("Optimal discrepancy reached: ", input.discrepancy(opt_group, group_complement(opt_group)))

AttributeError: 'function' object has no attribute 'discrepancy'

In [ ]:
output = (opt_group, group_complement(opt_group))